<a href="https://colab.research.google.com/github/filsto/GAN/blob/main/Simple_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building a simple Generative Adversarial Network using Tensorflow

In [ ]:
# Generating training data

import numpy as np

def get_y(x):
  return 10+x*x

def sample_data(n=10000, scale=100):
  data=[]

  x = scale*(np.random.random_sample((n,))-0.5)

  for i in range (n):
    yi = get_y(x[i])
    data.append(x[i], yi)

  return np.array(data)

In [ ]:
# Generator and discriminator networks implementation

def generator (Z, hsize=[16,16], reuse=False):
  with tf.variable_scope("GAN/Generator", reuse=reuse):
    h1 = tf.layers.Dense(Z, hsize[0], activation=tf.nn.leaky_relu)
    h2 = tf.layers.Dense(h1, hsize[1], activation=tf.nn.leaky_relu)
    out = tf.layers.Dense(h2,2)
  return out

In [ ]:
def discriminator(X, hsize=[16,16], reuse=False):
  with tf.variable_scope("GAN/Discriminator", reuse=reuse):
    h1 = tf.layers.Dense(X, hsize[0], activation=tf.nn.leaky_relu)
    h2 = tf.layers.Dense(h1, hsize[1], activation=tf.nn.leaky_relu)
    h3 = tf.layers.Dense(h2, 2)
    out = tf.layers.Dense(h3,1)
  return out, h3

In [ ]:
# Adversarial training

X = tf.placeholder(tf.float32, [None, 2])
Z = tf.placeholder(tf.float32, [None, 2])

In [ ]:
G_sample = generator(Z)
r_logits, r_rep = discriminator(X)
f_logits, g_rep = discriminator(G_sample, reuse=True)

In [ ]:
disc_loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits, labels=tf.one_like(r_logits)) + tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits, labels=tf.zeros_like(f_logits))
    )
gen_loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits, labels=tf.ones_like(f_logits))
    )

In [ ]:
gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Generator")
disc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Discriminator")

gen_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(gen_loss, var_list=gen_vars) # G train step
disc_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(disc_loss, var_list=disc_vars) # D train step

In [ ]:
for i in range(100001):
  X_batch = sample_data(n=batch_size)
  Z_batch = sample_Z(batch_size, 2)
  _, dloss = sess.run([disc_step, disc_loss], feed_dict={X:X_batch, Z: Z_batch})
  _, gloss = sess_run([gen_step, gen_loss], feed_dict={Z: Z_batch})

  print("Iterations : %d\t Discriminator loss: %.4f\t Generator loss: %.4f"%(i, dloss, gloss))